In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 

### 时间系列特征衍生

1、分类模型发现时间序列的特征重要性比较高，
进一步迭代运用回归模型预测时间序列特征---
重要特征 C1_pred_incre(mean,var,value),X_pred_var(sum),X3,X1,X4.

2、 对预测特征进一步加工衍生新的特征组；

In [2]:

train_df = pd.read_pickle('./data/feature_select_fill2_v1_15W.pkl')
print(train_df.shape)
train_df.head(2)

(152892, 1214)


,cust_no,lastq_I1,lastq_I2,lastq_I3,lastq_I4,lastq_I5,lastq_I6,lastq_I8,lastq_I10,lastq_I11,...,X_tou_incre / C1_divide_C2,lastq_B6_hour * X_sec_sum,lastq_C1_divide_lastq_C2 / C1_divide_C2_y,lastq_C1_incre / X_qq_cun_incre_mean_y,lastq_C1_sec * lastq_C1_std1mean,lastq_C1_sec / X_qq_cun_incre_mean,lastq_C1_sec / X_sec_sum,lastq_C1_std1mean / C1_std1mean_y,lastq_X3_std1mean / X3_incre_mean,lastq_X_cun_incre_v4 / X_qq_cun_incre_mean
0,0x3b9acb2b,男性,42.0,普通客户,0.0,NaN,1.0,天蝎座,NaN,0.0,...,NaN,22549540.4,0.920045,1.194786,33470.122224,-1.542867e+04,0.004037,0.993647,-33.025681,0.71186
1,0x3b9acc6c,女性,37.0,普通客户,0.0,NaN,1.0,天秤座,NaN,0.0,...,NaN,NaN,0.999041,1.339172,8736.139078,2.546472e+06,0.999032,0.999865,14821.363855,0.00000


In [4]:

def data_train(df, category_list, model_type):

    def to_category_or_num(df, category_list):
        for ft in df.columns:
            if ft in category_list:
                df[ft] = df[ft].astype('category')
            else:
                df[ft] = pd.to_numeric(df[ft], errors='coerce')
    
    
    # 训练集 预处理
    train_df_x = df[df.label.notnull()].drop(['cust_q','label','cust_no'],axis=1)
    to_category_or_num(train_df_x, category_list)

    # 划分标签y，特征x
    y = df[df.label.notnull()].label
    x = train_df_x

    #划分训练集，测试集
    from sklearn.model_selection import train_test_split

    train_x, test_x, train_y, test_y = train_test_split(x, y,test_size=0.2,random_state=0)
    train_df_x.head()


    # 评估
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,roc_curve,auc, cohen_kappa_score
    def model_metrics(model, x, y):
        yhat = model.predict(x)
        result = {'accuracy_score':accuracy_score(y, yhat),
                  'f1_score_macro': f1_score(y, yhat, average = "macro"),
                  'precision':precision_score(y, yhat,average="macro"),
                  'recall':recall_score(y, yhat,average="macro"),
                  'kappa': cohen_kappa_score(y, yhat)
                 }
        return result

    # 模型训练

    from xgboost import XGBClassifier
    from lightgbm import LGBMClassifier
    from catboost import CatBoostClassifier
    
    if model_type == 'lgb':
#         model = LGBMClassifier(bagging_fraction=0.7508393208895859, boosting_type='dart',
#                    class_weight='balanced', colsample_bytree=1.0,
#                    feature_fraction=0.7457350847503881, importance_type='split',
#                    lambda_l1=0.7905165383698197, lambda_l2=0.7836020789275822,
#                    learning_rate=0.28079595418448466, max_depth=10,
#                    min_child_samples=20, min_child_weight=1.1485987882882098e-05,
#                    min_data_in_leaf=5, min_split_gain=0.43016675319949704,
#                    n_estimators=415, n_jobs=-1, num_class=3, num_leaves=448,
#                    objective='multi:softprob', random_state=0, reg_alpha=0.0,
#                    reg_lambda=0.0, silent=True, subsample=1.0,
#                    subsample_for_bin=180000, subsample_freq=0, verbose=-1)
        model = LGBMClassifier(bagging_fraction=0.3520071162862265, boosting='dart',
               boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, feature_fraction=0.21860855317755723,
               importance_type='split', lambda_l1=0.25916787591548507,
               lambda_l2=0.07013352667143534, learning_rate=0.1344393208789723,
               max_depth=16, min_child_samples=20,
               min_child_weight=0.0003913820436320165, min_data_in_leaf=13,
               min_split_gain=0.37158177312975155, n_estimators=980, n_jobs=-1,
               num_leaves=1886, objective='multi:softprob', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=60000, subsample_freq=0, verbose=-1)

    
    elif model_type == 'xgb':
        model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1,n_estimators=500, n_jobs=-1,
              objective='multi:softprob', random_state=0,
              reg_alpha=1, reg_lambda=1, class_weight='balanced',
              silent=True, subsample=1, verbosity=1)
        
    elif model_type == 'cgb':
        model = CatBoostClassifier(auto_class_weights='Balanced',iterations=500, 
                                    max_depth=6,thread_count=-1,learning_rate=0.1,
                                  cat_features=category_list,loss_function='MultiClass',silent=True)

    model.fit(train_x, train_y)

    
    print('-'*30, 'train')
    print(model_metrics(model, train_x, train_y))

    print('-'*30, 'test')
    print(model_metrics(model, test_x, test_y))
    
    if model_type == 'lgb':
        # 重要特征
        import matplotlib.pyplot as plt
        from lightgbm import plot_importance,plot_tree

        plot_importance(model, max_num_features=50,figsize=(10,20),importance_type='split')
        plt.show()
        feature_importance = pd.DataFrame({
            'feature': model.booster_.feature_name(),
            'gain': model.booster_.feature_importance('gain'),
            'split': model.booster_.feature_importance('split')

        }).sort_values('gain',ascending=False)
        print(feature_importance[feature_importance.gain>0].values)
    
    return model,feature_importance

In [5]:
category_list = ['I1','I3','I5','I8','I10','I13','I14','lastq_I1','lastq_I3','lastq_I5','lastq_I8','lastq_I10','lastq_I13','lastq_I14'] 
# train_df3.loc[:,category_list] = train_df3.loc[:,category_list].fillna('-1')
# data_train(train_df3, category_list, 'cgb')
# df2 0.4885031208035543,  df3 0.48464523617323363  df 0.4857794193101631
# train_df2 = pd.read_pickle('./data/raw_features_fill2_v1_15W.pkl')
model2,feature_importance2 = data_train(train_df, category_list, 'lgb')

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.21860855317755723, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.21860855317755723
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] lambda_l1 is set=0.25916787591548507, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.25916787591548507
[LightGBM] [Warning] bagging_fraction is set=0.3520071162862265, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3520071162862265
[LightGBM] [Warning] lambda_l2 is set=0.07013352667143534, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.07013352667143534


KeyboardInterrupt: 

In [3]:
# 拼接cust q1的y
q1 = train_df[train_df.cust_q.str.contains('^q1'),~train_df.columns.str.contains('lastq')]
q341_df = train_df[train_df.cust_q.str.contains('^q4')].merge(q1, how='inner',left_on='cust_no',right_on='cust_no',suffixes=('','_y') )

print(q341_df.shape)
q341_df.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


(1697932, 177)


,B1,B1_fir,B1_fir_y,B1_sec,B1_sec_y,B1_y,B2,B2_fir,B2_fir_y,B2_sec,...,X8_sec,X8_sec_y,X8_y,cust_no,cust_q,cust_q_y,label,label_y,valid,valid_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0x3b9acb25,q30x3b9acb25,q40x3b9acb25,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0x3b9acb28,q30x3b9acb28,q40x3b9acb28,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0x3b9acb2a,q30x3b9acb2a,q40x3b9acb2a,NaN,NaN,NaN,NaN
3,5.0,1.0,4.0,3.0,1.0,4.0,2.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0x3b9acb2b,q30x3b9acb2b,q40x3b9acb2b,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0x3b9acb2c,q30x3b9acb2c,q40x3b9acb2c,NaN,NaN,NaN,NaN


In [1]:
import gc
gc.collect()
gc.enable()


NameError: name 'pd' is not defined

In [2]:
import pandas as pd
import gc
gc.collect()
gc.enable()


train_df2 = pd.read_csv('./data/q3q4_q4q1_filter_features_label.csv',index_col=0)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (31,32,48,49,52,53,54,55,57,60,61,64,65,66,68,70,71,72,74,75,76,77,78,79,80,81,82,84,85,89,90,91,92,93,104,110,114,120,173) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# # 建模及评估
def data_train(df, result_df,var_name,category_list, xdrop_list):

    def to_category_or_num(df, category_list):
        for ft in df.columns:
            if ft in category_list:
                df[ft] = df[ft].astype('category')
            else:
                df[ft] = pd.to_numeric(df[ft], errors='coerce')
    
    

    # 划分标签y，特征x, 处理类别/数值特征
    # 缺失剔除
    y = df.loc[(~df.cust_q.str.contains('^q1'))&df[var_name].notnull(),var_name].fillna(-1)

    x = df.loc[(~df.cust_q.str.contains('^q1'))&df[var_name].notnull(), ~df.columns.str.contains('_y$')].drop(xdrop_list,axis=1) # 不含y标签，不含testq1的特征变量

    oot_x = df.loc[:, ~df.columns.str.contains('_y$')].drop(xdrop_list,axis=1)

    to_category_or_num(x, category_list)
    to_category_or_num(oot_x, category_list)

    #划分训练集，测试集
    from sklearn.model_selection import train_test_split

    train_x, test_x, train_y, test_y = train_test_split(x, y,test_size=0.3,random_state=0)


    # 评估
    from sklearn.metrics import mean_squared_error
    def model_metrics(model, x, y):
        y_pred = model.predict(x)
        rmse = round(mean_squared_error(y_pred, y) ** 0.5, 5)
        return rmse

    # 模型训练
    from lightgbm import LGBMRegressor

    lgb = LGBMRegressor(random_state=1, reg_alpha=1, reg_lambda=1)
    lgb.fit(train_x, train_y,eval_set=(test_x, test_y),early_stopping_rounds=100,verbose=50)
    print(var_name,'\n')
    print('-'*30, 'train')
    print(model_metrics(lgb, train_x, train_y))

    print('-'*30, 'test')
    print(model_metrics(lgb, test_x, test_y))
    
    # 保存结果
    var = 'pred_' + var_name
    result_df[var] = lgb.predict(oot_x)
    


In [14]:
# 回归预测各特征变量；
result_df = train_df2.loc[:, ['cust_no','cust_q']]
train_df2.drop(drop_list,axis=1,inplace=True)
for var_name in train_df2.loc[:,train_df2.columns.str.contains('_y$')].columns.drop(['cust_q_y','valid_y']):
    try:
        data_train(train_df2, result_df, var_name, ['I1','I3','I5','I8','I10','I13','I14'], ['cust_no','cust_q','valid','label'])
    except Exception as e:
        print(e)
        continue
result_df.to_pickle('./data/all_regss_features_v1.pkl')
    
    

    
    
    
    
    

Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 215.231
[100]	valid_0's l2: 156.84
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 156.84
B1_fir_y 

------------------------------ train
11.81298
------------------------------ test
12.52358


D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 473.571
[100]	valid_0's l2: 320.897
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 320.897
B1_sec_y 

------------------------------ train
17.05355
------------------------------ test
17.91359


D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 753.874
[100]	valid_0's l2: 564.461
Did not meet early stopping. Best iteration is:
[99]	valid_0's l2: 564.062
B1_y 

------------------------------ train
22.71016
------------------------------ test
23.74999


...

Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 5.97559e+11
[100]	valid_0's l2: 4.46914e+11
Did not meet early stopping. Best iteration is:
[99]	valid_0's l2: 4.46739e+11
X8_sec_y 

------------------------------ train
662206.8861
------------------------------ test
668385.40302


D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 1.10872e+12
[100]	valid_0's l2: 9.14867e+11
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 9.14867e+11
X8_y 

------------------------------ train
941505.99806
------------------------------ test
956487.018


D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[50]	valid_0's l2: 0.609916
[100]	valid_0's l2: 0.599459
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.599459
label_y 

------------------------------ train
0.73689
------------------------------ test
0.77425


In [21]:
result_df.head()

,cust_no,cust_q,pred_B1_fir_y,pred_B1_sec_y,pred_B1_y,pred_B2_fir_y,pred_B2_sec_y,pred_B2_y,pred_B3_fir_y,pred_B3_sec_y,...,pred_X6_fir_y,pred_X6_sec_y,pred_X6_y,pred_X7_fir_y,pred_X7_sec_y,pred_X7_y,pred_X8_fir_y,pred_X8_sec_y,pred_X8_y,pred_label_y
3,0x3b9acb2b,q30x3b9acb2b,3.991641,7.869153,11.421230,3.699205,-3.637694,6.430688,593462.757464,569688.332705,...,89948.281886,50636.05833,106886.856527,9.768966e+05,9.846634e+05,9.692336e+05,479569.955765,391870.221833,626093.795679,0.272402
12,0x3b9acc6c,q30x3b9acc6c,7.797056,0.970903,12.567592,-0.123236,-9.130609,-9.967883,539455.786571,569688.332705,...,89948.281886,50636.05833,106886.856527,4.647051e+06,1.955709e+06,3.203140e+06,502478.446647,467665.868385,653798.151050,0.687818
20,0x3b9acc87,q30x3b9acc87,6.354636,13.011314,22.704454,5.239021,1.347244,13.313368,539455.786571,569688.332705,...,89948.281886,50636.05833,106886.856527,7.869048e+05,1.211395e+06,2.869117e+06,267479.827763,263246.547008,369349.175636,-0.117612
26,0x3b9acca4,q30x3b9acca4,5.124919,1.773302,23.347812,2.212429,-16.851508,13.733721,539455.786571,569688.332705,...,89948.281886,50636.05833,106886.856527,8.757881e+05,8.617939e+05,3.598345e+06,164494.358936,206929.834057,474229.536406,0.306030
35,0x3b9accb7,q30x3b9accb7,6.067408,1.081519,12.825306,-6.328761,6.915839,10.720769,539455.786571,569688.332705,...,89948.281886,50636.05833,106886.856527,4.320868e+06,2.078440e+06,3.731413e+06,111606.908148,208676.477708,200483.520988,0.732253


In [23]:
train_df2.head()

,B1,B1_fir,B1_fir_y,B1_sec,B1_sec_y,B1_y,B2,B2_fir,B2_fir_y,B2_sec,...,E14_dayofweek,E16_kaiinterval,E16_interval,E16_dayofweek,E18_kaiinterval,E18_interval,E18_dayofweek,E15_reduce_E17,E15_ratio,E17_ratio
3,5.0,1.0,4.0,3.0,1.0,4.0,2.0,2.0,3.0,2.0,...,NaN,NaN,NaN,NaN,5281.0,-1180.0,4.0,-12500.00,0.0,0.358544
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,NaN,NaN,NaN,4928.0,-1114.0,0.0,-50000.00,NaN,inf
20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-675.00,NaN,inf
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,4656.0,-1314.0,3.0,-60202.39,NaN,inf
